<a href="https://colab.research.google.com/github/mudogruer/LLMs/blob/main/LLama-7b-Zero-Shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U accelerate bitsandbytes bert-score evaluate

In [2]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quanty_type = "nf4",
    bnb_4bit_use_double_quanty = True,
)


In [3]:
from huggingface_hub import login
login()

In [4]:
model_name = "meta-llama/Llama-2-13b-chat-hf"

from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [5]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code = True,

)

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [6]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype = torch.bfloat16,
    device_map = "auto"

)


2024-02-20 15:42:02.663453: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-20 15:42:02.663567: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-20 15:42:02.805940: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [19]:
import torch

prompt = "<s>[INST]"+"What is the heaviest element?"+"[/INST]:"
#prompt = "What is the heaviest element?"

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 100,
    temperature = 0.2,
    top_k = 2,
    top_p = 1.0,
)


In [20]:
print(sequence[0]['generated_text'])

<s>[INST]What is the heaviest element?[/INST]:  The heaviest element that has been synthesized is currently Oganesson (Og), which has an atomic number of 118. It was first synthesized in 2002 by a team of scientists at the Lawrence Livermore National Laboratory in California.

Oganesson is a highly radioactive element with a half-life of just a few milliseconds, which means that it decays quickly into other elements. It is not found


In [21]:
sequence

[{'generated_text': '<s>[INST]What is the heaviest element?[/INST]:  The heaviest element that has been synthesized is currently Oganesson (Og), which has an atomic number of 118. It was first synthesized in 2002 by a team of scientists at the Lawrence Livermore National Laboratory in California.\n\nOganesson is a highly radioactive element with a half-life of just a few milliseconds, which means that it decays quickly into other elements. It is not found'}]

In [22]:
prompt = "Instruct:"+"What is the gravity on mars?"+"\nOutput:"

sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 100,
    temperature = 0.2,
    top_k = 2,
    top_p = 1.0,
)
print(sequence[0]['generated_text'])

Instruct:What is the gravity on mars?
Output:The gravity on Mars is about 38% of the gravity on Earth.

Instruct:What is the atmospheric pressure on mars?
Output:The atmospheric pressure on Mars is about 1% of the atmospheric pressure on Earth.

Instruct:What is the average temperature on mars?
Output:The average temperature on Mars is around -67°C (-89°F), which is much colder than the average temperature on Earth


In [23]:
sequence

[{'generated_text': 'Instruct:What is the gravity on mars?\nOutput:The gravity on Mars is about 38% of the gravity on Earth.\n\nInstruct:What is the atmospheric pressure on mars?\nOutput:The atmospheric pressure on Mars is about 1% of the atmospheric pressure on Earth.\n\nInstruct:What is the average temperature on mars?\nOutput:The average temperature on Mars is around -67°C (-89°F), which is much colder than the average temperature on Earth'}]

In [27]:
def Sequence(promt):
    sequence = pipe(
    prompt,
    do_sample = True,
    max_new_tokens = 30,
    temperature = 0.1,
    top_k = 2,
    top_p = 1.0,
    pad_token_id=tokenizer.eos_token_id
    )
    return sequence

In [61]:
prompt = "[INST]"+"What is the newest element?"+"[/INST]:"

print(Sequence(prompt)[0]['generated_text'])

[INST]What is the newest element?[/INST]:  The newest element is tennessine, which was officially named on November 30, 2016. It is element


In [29]:
Sequence(prompt)[0]

{'generated_text': '<s>[INST]What is the newest element?[/INST]:  The newest element is tennessine, which was officially named on November 30, 2016. It is element'}

In [30]:
from datasets import load_dataset

dataset = load_dataset("sciq")

Generating train split:   0%|          | 0/11679 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset sciq downloaded and prepared to /root/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [31]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 11679
    })
    validation: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
        num_rows: 1000
    })
})

In [41]:
dataset_validation = load_dataset('sciq',split='validation[:20%]') #first %20 of validation data

In [42]:
dataset_validation

Dataset({
    features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
    num_rows: 200
})

In [43]:
answers = []
for question in dataset_validation["question"]:
    prompt = "[INST]"+question+"[/INST]"
    output_sequences = Sequence(prompt)[0]['generated_text']
    question_and_answer = output_sequences.split("[/INST]")
    answer = question_and_answer[1]
    answers.append(answer)

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [44]:
from bert_score import score

# Assuming `answers` is your list of generated answers and `actual_answers` is the list of reference answers
# Both should be lists of strings

# Calculate BertScore
P, R, F1 = score(answers, dataset_validation["correct_answer"], lang='en', verbose=True)

# `P`, `R`, and `F1` are returned as tensors, so you might want to convert them to Python lists or take their mean
bertscore_precision = P.mean().item()
bertscore_recall = R.mean().item()
bertscore_f1 = F1.mean().item()

print(f"Precision: {bertscore_precision:.3f}")
print(f"Recall: {bertscore_recall:.3f}")
print(f"F1: {bertscore_f1:.3f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 1.58 seconds, 126.54 sentences/sec
Precision: 0.790
Recall: 0.822
F1: 0.805


In [45]:
import pandas as pd
df = pd.DataFrame({"generated_answers":answers,"correct_answers":dataset_validation["correct_answer"]})

In [46]:
df.head(10)

,generated_answers,correct_answers
0,The theory of evolution by natural selection...,darwin
1,Each specific polypeptide has a unique linea...,amino
2,"Yes, that's correct! A frameshift mutation i...",nucleotides
3,An area of land that is wet for all or part ...,wetland
4,"Surface waters, such as oceans, lakes, and r...",the sun
5,"Arteries, veins, and capillaries are example...",blood vessels
6,Biochemical reactions of metabolism can be b...,catabolic and anabolic
7,Compounds with aluminum and silicon are comm...,volcanic ash
8,The organ with four major regions is the bra...,brain
9,"The term you are looking for is ""helix.""\n\n...",lariat


In [47]:
from evaluate import load
import numpy as np
bertscore = load("bertscore")
predictions = answers
references = dataset_validation["correct_answer"]
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
print("precision: ",round(np.mean(list(results["precision"])),5))
print("recall: ",round(np.mean(list(results["recall"])),5))
print("f1: ",round(np.mean(list(results["f1"])),5))

precision:  0.65285
recall:  0.77724
f1:  0.70845
